In [1]:
import argparse
from typing import Dict
from ray.air import session
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import ray.train as train
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="~/data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="~/data",
    train=False,
    download=True,
    transform=ToTensor(),
)


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) // session.get_world_size()
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def validate_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) // session.get_world_size()
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n "
        f"Accuracy: {(100 * correct):>0.1f}%, "
        f"Avg loss: {test_loss:>8f} \n"
    )
    return test_loss


def train_func(config: Dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]

    worker_batch_size = batch_size // session.get_world_size()

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=worker_batch_size)
    test_dataloader = DataLoader(test_data, batch_size=worker_batch_size)

    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)

    # Create model.
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for _ in range(epochs):
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        loss = validate_epoch(test_dataloader, model, loss_fn)
        session.report(dict(loss=loss))


def train_fashion_mnist(num_workers=3, use_gpu=True):
    trainer = TorchTrainer(
        train_loop_per_worker=train_func,
        train_loop_config={"lr": 1e-3, "batch_size": 64, "epochs": 4},
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    )
    result = trainer.fit()
    print(f"Last result: {result.metrics}")

In [3]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--address", required=False, type=str, help="the address to use for Ray"
    )
    parser.add_argument(
        "--num-workers",
        "-n",
        type=int,
        default=3,
        help="Sets number of workers for training.",
    )
    parser.add_argument(
        "--use-gpu", action="store_true", default=True, help="Enables GPU training"
    )
    parser.add_argument(
        "--smoke-test",
        action="store_true",
        default=False,
        help="Finish quickly for testing.",
    )

    args, _ = parser.parse_known_args()

    import ray
    if ray.is_initialized() == False:
        print("Connecting to Ray cluster...")
        service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
        service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
        ray.util.connect(f"{service_host}:{service_port}")


    # if args.smoke_test:
    #     # 2 workers + 1 for trainer.
    #     ray.init(num_cpus=3)
    #     train_fashion_mnist()
    # else:
    # ray.init(address=args.address)
    train_fashion_mnist(num_workers=args.num_workers, use_gpu=True)

Connecting to Ray cluster...


/opt/conda/lib/python3.9/site-packages/ray/util/client/worker.py:597: UserWarning: More than 10MB of messages have been created to schedule tasks on the server. This can be slow on Ray Client due to communication overhead over the network. If you're running many fine-grained tasks, consider running them inside a single remote function. See the section on "Too fine-grained tasks" in the Ray Design Patterns document for more details: https://docs.google.com/document/d/167rnnDFIVRhHhK4mznEIemOtj63IOhtIPvSYaPgI4Fg/edit#heading=h.f7ins22n6nyl. If your functions frequently use large objects, consider storing the objects remotely with ray.put. An example of this is shown in the "Closure capture of large / unserializable object" section of the Ray Design Patterns document, available here: https://docs.google.com/document/d/167rnnDFIVRhHhK4mznEIemOtj63IOhtIPvSYaPgI4Fg/edit#heading=h.1afmymq455wu
  warnings.warn(


(TunerInternal pid=2109) == Status ==
(TunerInternal pid=2109) Current time: 2023-02-24 16:29:22 (running for 00:00:06.37)
(TunerInternal pid=2109) Memory usage on this node: 5.2/59.9 GiB
(TunerInternal pid=2109) Using FIFO scheduling algorithm.
(TunerInternal pid=2109) Resources requested: 1.0/8 CPUs, 3.0/4 GPUs, 0.0/26.63 GiB heap, 0.0/11.83 GiB objects (0.0/4.0 accelerator_type:V100)
(TunerInternal pid=2109) Result logdir: /home/ray/ray_results/TorchTrainer_2023-02-24_16-29-14
(TunerInternal pid=2109) Number of trials: 1/1 (1 RUNNING)
(TunerInternal pid=2109) +--------------------------+----------+----------------+
(TunerInternal pid=2109) | Trial name               | status   | loc            |
(TunerInternal pid=2109) |--------------------------+----------+----------------|
(TunerInternal pid=2109) | TorchTrainer_6ef96_00000 | RUNNING  | 10.0.53.75:217 |
(TunerInternal pid=2109) +--------------------------+----------+----------------+
(TunerInternal pid=2109) 
(TunerInternal pid=2

(RayTrainWorker pid=135, ip=10.0.52.196) 2023-02-24 16:29:24,928	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=3]


(TunerInternal pid=2109) == Status ==
(TunerInternal pid=2109) Current time: 2023-02-24 16:29:27 (running for 00:00:11.37)
(TunerInternal pid=2109) Memory usage on this node: 6.2/59.9 GiB
(TunerInternal pid=2109) Using FIFO scheduling algorithm.
(TunerInternal pid=2109) Resources requested: 1.0/8 CPUs, 3.0/4 GPUs, 0.0/26.63 GiB heap, 0.0/11.83 GiB objects (0.0/4.0 accelerator_type:V100)
(TunerInternal pid=2109) Result logdir: /home/ray/ray_results/TorchTrainer_2023-02-24_16-29-14
(TunerInternal pid=2109) Number of trials: 1/1 (1 RUNNING)
(TunerInternal pid=2109) +--------------------------+----------+----------------+
(TunerInternal pid=2109) | Trial name               | status   | loc            |
(TunerInternal pid=2109) |--------------------------+----------+----------------|
(TunerInternal pid=2109) | TorchTrainer_6ef96_00000 | RUNNING  | 10.0.53.75:217 |
(TunerInternal pid=2109) +--------------------------+----------+----------------+
(TunerInternal pid=2109) 
(TunerInternal pid=2

(RayTrainWorker pid=135, ip=10.0.52.196) 2023-02-24 16:29:29,240	INFO train_loop_utils.py:300 -- Moving model to device: cuda:0
(RayTrainWorker pid=135, ip=10.0.52.196) 2023-02-24 16:29:29,242	INFO train_loop_utils.py:347 -- Wrapping provided model in DDP.
(RayTrainWorker pid=2208) 2023-02-24 16:29:29,257	INFO train_loop_utils.py:300 -- Moving model to device: cuda:0
(RayTrainWorker pid=2208) 2023-02-24 16:29:29,258	INFO train_loop_utils.py:347 -- Wrapping provided model in DDP.
(RayTrainWorker pid=246, ip=10.0.53.75) 2023-02-24 16:29:29,380	INFO train_loop_utils.py:300 -- Moving model to device: cuda:0
(RayTrainWorker pid=246, ip=10.0.53.75) 2023-02-24 16:29:29,382	INFO train_loop_utils.py:347 -- Wrapping provided model in DDP.


(RayTrainWorker pid=246, ip=10.0.53.75) loss: 2.300348  [    0/20000]
(RayTrainWorker pid=135, ip=10.0.52.196) loss: 2.304214  [    0/20000]
(RayTrainWorker pid=2208) loss: 2.296877  [    0/20000]
(RayTrainWorker pid=2208) loss: 2.283382  [ 2100/20000]
(RayTrainWorker pid=246, ip=10.0.53.75) loss: 2.280409  [ 2100/20000]
(RayTrainWorker pid=135, ip=10.0.52.196) loss: 2.293032  [ 2100/20000]
(TunerInternal pid=2109) == Status ==
(TunerInternal pid=2109) Current time: 2023-02-24 16:29:32 (running for 00:00:16.38)
(TunerInternal pid=2109) Memory usage on this node: 7.6/59.9 GiB
(TunerInternal pid=2109) Using FIFO scheduling algorithm.
(TunerInternal pid=2109) Resources requested: 1.0/8 CPUs, 3.0/4 GPUs, 0.0/26.63 GiB heap, 0.0/11.83 GiB objects (0.0/4.0 accelerator_type:V100)
(TunerInternal pid=2109) Result logdir: /home/ray/ray_results/TorchTrainer_2023-02-24_16-29-14
(TunerInternal pid=2109) Number of trials: 1/1 (1 RUNNING)
(TunerInternal pid=2109) +--------------------------+---------

(TunerInternal pid=2109) 2023-02-24 16:30:37,822	WARNING util.py:220 -- The `process_trial_save` operation took 2.431 s, which may be a performance bottleneck.
(TunerInternal pid=2109) 2023-02-24 16:30:37,822	WARNING trial_runner.py:950 -- Consider turning off forced head-worker trial checkpoint syncs by setting sync_on_checkpoint=False. Note that this may result in faulty trial restoration if a failure occurs while the checkpoint is being synced from the worker to the head node.
2023-02-25 00:30:39,215	ERROR checkpoint_manager.py:133 -- The requested checkpoint is not available on this node, most likely because you are using Ray client or disabled checkpoint synchronization. To avoid this, enable checkpoint synchronization to cloud storage by specifying a `SyncConfig`. The checkpoint may be available on a different node - please check this location on worker nodes: /home/ray/ray_results/TorchTrainer_2023-02-24_16-29-14/TorchTrainer_6ef96_00000_0_2023-02-24_16-29-17/checkpoint_-00001


(TunerInternal pid=2109) == Status ==
(TunerInternal pid=2109) Current time: 2023-02-24 16:30:39 (running for 00:01:23.13)
(TunerInternal pid=2109) Memory usage on this node: 4.7/59.9 GiB
(TunerInternal pid=2109) Using FIFO scheduling algorithm.
(TunerInternal pid=2109) Resources requested: 0/8 CPUs, 0/4 GPUs, 0.0/26.63 GiB heap, 0.0/11.83 GiB objects (0.0/4.0 accelerator_type:V100)
(TunerInternal pid=2109) Result logdir: /home/ray/ray_results/TorchTrainer_2023-02-24_16-29-14
(TunerInternal pid=2109) Number of trials: 1/1 (1 TERMINATED)
(TunerInternal pid=2109) +--------------------------+------------+----------------+--------+------------------+---------+--------------+---------------------+
(TunerInternal pid=2109) | Trial name               | status     | loc            |   iter |   total time (s) |    loss |   _timestamp |   _time_this_iter_s |
(TunerInternal pid=2109) |--------------------------+------------+----------------+--------+------------------+---------+--------------+---

(TunerInternal pid=2109) 2023-02-24 16:30:39,200	INFO tune.py:758 -- Total run time: 84.87 seconds (83.12 seconds for the tuning loop).
